In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def getPageHtml(url):
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    #options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
    
    # Chrome WebDriver 실행
    driver = webdriver.Chrome('chromedriver', options=options)

    # 페이지 열기
    driver.get(url)

    # 페이지의 초기 높이 가져오기
    scroll_pause_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # 페이지 충분히 스크롤
    scroll_pause_time = 0.5  # 스크롤링 사이의 일시 정지 시간 (초)
    while True:
        # 스크롤을 아래로 이동
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # 일시 정지
        time.sleep(scroll_pause_time)

        # 새로운 스크롤 높이 가져오기
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")

        # 더 이상 스크롤링이 되지 않으면 종료
        if new_scroll_height == scroll_pause_height:
            break

        # 스크롤 높이 업데이트
        scroll_pause_height = new_scroll_height

    # 페이지의 HTML 가져오기
    html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # WebDriver 종료
    driver.quit()

    return soup



def getUrl(dfrow):
    
    row = dfrow.copy()
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    if row['mall'] != '쿠팡':
        options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")    
    
    url = row['링크']

    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    #리디렉션 완료될 때까지 기다리기
    while 'https://cr.shopping.naver.com/'in driver.current_url:
        time.sleep(0)
    time.sleep(.1)
    html = driver.page_source
    source = BeautifulSoup(html, "html.parser")
    nlink = driver.current_url

    driver.quit()

    return (source, nlink)

In [11]:
urlList = ["https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=1&pagingSize=80&productSet=total&query=D10XPA&sort=price_asc&timestamp=&viewType=list",
           "https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=2&pagingSize=80&productSet=total&query=D10XPA&sort=price_asc&timestamp=&viewType=list"
           ]
           
alist = []
for url in urlList:
    soup = getPageHtml(url)
    tag = soup.select_one('#content > div.style_content__xWg5l > div.basicList_list_basis__uNBZx')
    alist += tag.select('div[class*="product_item__"]')


In [12]:
def get_prod_list_df(alist):
    import pandas as pd
    import re
    rowdata = []
    for i in alist:
        if i.select_one('span[class*="price_delivery"]'):
            title = i.select_one('div[class*="title"] > a[href]')
            link = title['href']
            title = title.text
            mall = i.select_one('div[class*="mall_title"] > a[class*="mall"]')
            shopname = mall.text
            if not shopname:
                mallname = mall.select_one('img[alt]')['alt']
            
            price = i.select_one('span[class*="price_price"]').text    
            price = int(re.sub(r'[^0-9]', '', price))
            delivery = i.select_one('span[class*="price_delivery"]').text    
            delivery = re.sub(r'[^0-9]', '', delivery)
            if delivery=='':
                delivery = 0;
            else:
                delivery = int(delivery)

            sum = int(price)+int(delivery)
            row = {'mall':mallname, 'shop':shopname, '제목':title, '가격':price, '배송비':delivery, '합계':sum, '링크':link}
            rowdata.append(row)
    df = pd.DataFrame(rowdata)
    df['수량'] = df['제목'].str.extract(r'([0-9]+)개')
    df = df.fillna(1)
    df['수량']=df['수량'].astype(int)
    df['단가'] = df['가격']/df['수량']
    df['위반'] = df['단가']<31000
    df = df[df['위반']]
    return df
df = get_prod_list_df(alist)
df

,mall,shop,제목,가격,배송비,합계,링크,수량,단가,위반
0,G마켓,,팬직 하늘풍력/팬직/천장용팬/실링팬/환풍기/TFV-D10XPA,25560,5000,30560,https://cr.shopping.naver.com/adcr.nhn?x=o64%2...,1,25560.000000,True
1,옥션,,하늘풍력/팬직/천장용팬/실링팬/환풍기/TFV-D10XPA,25830,4000,29830,https://cr.shopping.naver.com/adcr.nhn?x=Ty7VX...,1,25830.000000,True
2,옥션,,욕실 환풍기 팬직 TFV-D10XPA 화장실 천장용 환기팬,26080,2500,28580,https://cr.shopping.naver.com/adcr.nhn?x=7pjCB...,1,26080.000000,True
3,11번가,,TFV-D10XPA 팬직 욕실환풍기 화장실환풍기 댐퍼장착,26090,0,26090,https://cr.shopping.naver.com/adcr.nhn?x=6kGl4...,1,26090.000000,True
4,쿠팡,,팬직 천장형 욕실 화장실 환풍기 배기팬 D10XPA 1개,26400,4000,30400,https://cr.shopping.naver.com/adcr.nhn?x=mP3%2...,1,26400.000000,True
5,G마켓,,하늘풍력/팬직/천장용팬/실링팬/환풍기/TFV-D10XPA,26760,3000,29760,https://cr.shopping.naver.com/adcr.nhn?x=vPrL%...,1,26760.000000,True
6,옥션,,하늘풍력/팬직/천장용팬/실링팬/환풍기/TFV-D10XPA,26770,3500,30270,https://cr.shopping.naver.com/adcr.nhn?x=xI3Jw...,1,26770.000000,True
7,옥션,대성풍력,[팬직] 천정용 플라스틱 환풍기 욕실용환풍기 사무실환풍기 TFV-D10XPA,26900,3500,30400,https://cr.shopping.naver.com/adcr.nhn?x=%2Frl...,1,26900.000000,True
8,옥션,,팬직 국산정품/욕실환풍기/주방화장실용/TFV-D10XPA,27200,0,27200,https://cr.shopping.naver.com/adcr.nhn?x=z%2FS...,1,27200.000000,True
9,11번가,,팬직 욕실용 환풍기 화장실 저소음 팬 아파트 설치 교체 TFV-D10XPA,27280,0,27280,https://cr.shopping.naver.com/adcr.nhn?x=z9Ic1...,1,27280.000000,True


In [ ]:
cdf = df[df['mall']=='쿠팡']
gdf = df[df['mall']=='G마켓']
edf = df[df['mall']=='11번가']
adf = df[df['mall']=='옥션']


In [1]:
#
def get_mall_name(df):
    import pandas as pd
    selector = {'쿠팡':'div[class="prod-sale-vendor"] a', 'G마켓':'span[class*="seller"]', '11번가':'h1[class*="store_title"]', '옥션':'span[class*="text__seller"]'}
    length = str(len(df))
    print(length+'개의 목록 작업 시작')
    rlist = []
    for i in range(len(df)):
        print(str(i+1)+' 번째')
        row = df.iloc[i].copy()
        if row['mall'] in ['쿠팡', 'G마켓', '11번가', '옥션']:
            print(row['mall'])
            (soup, nlink) = getUrl(row)
            shopInfo = soup.select_one(selector[row['mall']])
            if shopInfo:
                shop = shopInfo.text.strip()
            else:
                shop = ''
            row['링크'] = nlink
            row['platform'] = row['mall']
            row['mall'] = shop
        rlist.append(row)
    ndf = pd.DataFrame(rlist)

    return ndf

df2 = get_mall_name(df)
df2

NameError: name 'df' is not defined

In [ ]:
df2.to_excel('list.xlsx', index=False)